# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [16]:
# All import statements needed for the project, for example:
import math
from datetime import datetime
from datetime import date
import numpy as np
import json
import pathlib
import urllib.parse
import psycopg2
import psycopg2.extras
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from pyproj import CRS, Transformer
from shapely.ops import transform
import seaborn as sns
import pandas as pd
import requests
import shapely
import sqlalchemy as db
import os
from shapely.geometry import Point
from sqlalchemy.orm import declarative_base

In [17]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

APP_TOKEN = "noFU7vdLMu3RKtmWOyOqkOi2x"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.json"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "fg_database"
DB_USER = "postgres"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

crs='EPSG:4326'

rent_month_dict = {}
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [18]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [14]:
def download_nyc_311_data(url, start, end, limit, force=False):

    url_path = urllib.parse.urlparse(url).path.split('/')[-1]
    filename = DATA_DIR / url_path
    
    if not DATA_DIR.exists():
        DATA_DIR.mkdir()

    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        offset = start
        all_entries = []
        selected_columns = ["unique_key", "created_date", "complaint_type", "incident_zip", "location"]
        
        while offset + limit <= end:  
            total = limit+offset
            print(f"start from {offset} and end is {total}")
            soql_query = f"{url}?$$app_token={APP_TOKEN}&$select={','.join(selected_columns)}&$limit={limit}&$offset={offset}"
            response = requests.get(soql_query)
            if response.status_code == 200: 
                entries = response.json()
                if not entries:
                    break 
                all_entries.extend(entries)
                offset += limit
            else:
                print(f"cannot access url error code{response.status_code}")
                break
        
        
        with open(filename, "w") as f:
            json.dump(all_entries, f)
        print(f"Done downloading {url}.")

    else:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(f"Current Time = {current_time} Reading from {filename}...")
    
    
    return filename

In [15]:
def download_nyc_tree_data(url, force=False):
    
    url_path = urllib.parse.urlparse(url).path.split('/')[-1]
    filename = DATA_DIR / url_path
    
    if not DATA_DIR.exists():
        DATA_DIR.mkdir()

    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        limit = 1_000_000
        offset = 0
        all_entries = []
        selected_columns = ["created_at", "tree_id", "zipcode", "the_geom", "spc_common", "health", "status"]
        end = 1_000_000
        
        while offset < end:  
            total = limit+offset
            print(f"start from {offset} and total is {total}")
            soql_query = f"{url}?$$app_token={APP_TOKEN}&$select={','.join(selected_columns)}&$limit={limit}&$offset={offset}"
            response = requests.get(soql_query)
            if response.status_code == 200: 
                entries = response.json()
                if not entries:
                    break 
                all_entries.extend(entries)
                offset += limit
            else:
                print(f"cannot access url error code{response.status_code}")
                break
        
        
        with open(filename, "w") as f:
            json.dump(all_entries, f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [16]:
def load_and_clean_zipcodes(zipcode_datafile):
    global unique_zipcodes
    gdf = gpd.read_file("data/nyc_zipcodes/nyc_zipcodes.shp")
    gdf = gdf.to_crs(4326)
    zip_code = gdf[["ZIPCODE", "geometry"]].copy()
    unique_zipcodes_df = zip_code.drop_duplicates(subset = ["ZIPCODE"], keep = "last").reset_index()
    unique_zipcodes = unique_zipcodes_df["ZIPCODE"]
    return unique_zipcodes_df

In [17]:
def download_and_clean_311_data():
    Service_Requests_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_311}"
    start = 0
    end = 100_000_000
    limit = 1_000_000
    filename = download_nyc_311_data(Service_Requests_url, start, end, limit)
    
    interactions_geo_data_frame = gpd.GeoDataFrame(pd.read_json(filename), dtype='object')
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(f"Current Time = {current_time} reading done from {filename}...")`
    
    if 'location' in interactions_geo_data_frame.columns:
            interactions_geo_data_frame['longitude'] = interactions_geo_data_frame['location'].apply(
                lambda loc: loc.get('longitude') if isinstance(loc, dict) else None
            )
            interactions_geo_data_frame['latitude'] = interactions_geo_data_frame['location'].apply(
                lambda loc: loc.get('latitude') if isinstance(loc, dict) else None
            )
            
            geometry = gpd.GeoSeries(
                interactions_geo_data_frame.apply(
                    lambda row: Point(float(row['longitude']), float(row['latitude'])) if not pd.isna(row['longitude']) and not pd.isna(row['latitude']) else None,
                    axis=1
                ),
                crs='EPSG:4326'
            )
    
    if 'location' in interactions_geo_data_frame.columns:
        interactions_geo_data_frame.drop(columns=['location'], inplace=True)
    if 'longitude' in interactions_geo_data_frame.columns:
        interactions_geo_data_frame.drop(columns=['longitude'], inplace=True)
    if 'latitude' in interactions_geo_data_frame.columns:
        interactions_geo_data_frame.drop(columns=['latitude'], inplace=True)
            
    interactions_geo_data_frame.set_geometry(geometry, inplace=True)
        
    interactions_geo_data_frame = interactions_geo_data_frame.dropna(subset=['incident_zip'])
    interactions_geo_data_frame = interactions_geo_data_frame[interactions_geo_data_frame["incident_zip"].isin(unique_zipcodes)]
    
    interactions_geo_data_frame['created_date'] = pd.to_datetime(interactions_geo_data_frame['created_date'], format='%Y-%m-%dT%H:%M:%S.%f')
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(f"Current Time = {current_time} done")
    
    #interactions_geo_df = interactions_geo_df.drop_duplicates(subset=['unique_key']).reset_index(drop=True)
    
    
    
    return interactions_geo_data_frame

In [18]:
def load_local_311_data():
    # Directory containing the JSON files
    json_files_directory = 'complain_data/'
    json_files = [f"{i}.json" for i in range(1, 36)]

    interactions_geo_data_frames = []

    # Iterate through each file in the directory
    for interactions_input_file in json_files:
        now = datetime.now()

        current_time = now.strftime("%H:%M:%S")
        print(f"Current Time = {current_time} input_file {interactions_input_file} begin" )
        file_path = os.path.join(json_files_directory, interactions_input_file)
        interactions_geo_data_frame = gpd.GeoDataFrame(pd.read_json(file_path, dtype='object'))
        #interactions_geo_data_frame = gpd.read_file(file_path, dtype='object')
    
        if 'location' in interactions_geo_data_frame.columns:
            #if 'longitude' in interactions_geo_data_frame['location'].columns and 'latitude' in interactions_geo_data_frame['location'].columns:
                interactions_geo_data_frame['longitude'] = interactions_geo_data_frame['location'].apply(
                    lambda loc: loc.get('longitude') if isinstance(loc, dict) else None
                )
                interactions_geo_data_frame['latitude'] = interactions_geo_data_frame['location'].apply(
                    lambda loc: loc.get('latitude') if isinstance(loc, dict) else None
                )


                geometry = gpd.GeoSeries(
                    interactions_geo_data_frame.apply(
                        lambda row: Point(float(row['longitude']), float(row['latitude'])) if not pd.isna(row['longitude']) and not pd.isna(row['latitude']) else None,
                        axis=1
                    ),
                    crs='EPSG:4326'
                )

        

        if 'location' in interactions_geo_data_frame.columns:
                interactions_geo_data_frame.drop(columns=['location'], inplace=True)
        if 'longitude' in interactions_geo_data_frame.columns:
                interactions_geo_data_frame.drop(columns=['longitude'], inplace=True)
        if 'latitude' in interactions_geo_data_frame.columns:
                interactions_geo_data_frame.drop(columns=['latitude'], inplace=True)
    
        interactions_geo_data_frame.set_geometry(geometry, inplace=True)
        
        interactions_geo_data_frame = interactions_geo_data_frame.dropna(subset=['incident_zip'])
        interactions_geo_data_frame = interactions_geo_data_frame[interactions_geo_data_frame["incident_zip"].isin(unique_zipcodes)]

        interactions_geo_data_frame['created_date'] = pd.to_datetime(interactions_geo_data_frame['created_date'], format='%Y-%m-%dT%H:%M:%S.%f')

        interactions_geo_data_frames.append(interactions_geo_data_frame)
    
    
    current_time = now.strftime("%H:%M:%S")
    print(f"Current Time = {current_time} load file done" )
    interactions_geo_df = gpd.GeoDataFrame(pd.concat(interactions_geo_data_frames, ignore_index=True, sort=True))
    current_time = now.strftime("%H:%M:%S")
    print(f"Current Time = {current_time} concat file done" )
    interactions_geo_df.crs = interactions_geo_data_frames[0].crs
    interactions_geo_data_frames.clear()
    interactions_geo_df = interactions_geo_df.drop_duplicates(subset=['unique_key']).reset_index(drop=True)
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print(f"Current Time = {current_time} done normalized data 311" )
    
    return interactions_geo_df

In [19]:
def download_and_clean_tree_data():
    Trees_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}"
    filename = download_nyc_tree_data(Trees_url)
    trees_gdf = gpd.GeoDataFrame(pd.read_json(filename), dtype='object')
    convert_dict = {
        "tree_id" : int,
        "spc_common" :str,
        "health": str,
        "status": str,
        "the_geom" : "geometry",
        "zipcode":str
    }
    
    trees_gdf['the_geom'] = gpd.GeoSeries(
        trees_gdf['the_geom'].apply(lambda x: Point(x['coordinates'])), crs='EPSG:4326'
    )
    
    trees_gdf_normalized = gpd.GeoDataFrame(trees_gdf.astype(convert_dict))
    trees_gdf_normalized = gpd.GeoDataFrame(trees_gdf_normalized[trees_gdf_normalized["zipcode"].isin(unique_zipcodes)])
    
    return trees_gdf_normalized

In [20]:
def load_and_clean_zillow_data(ZILLOW_DATA_FILE):
    global rent_month_dict
    zillow_rent_data = pd.read_csv(ZILLOW_DATA_FILE)
    NY_rent_date = zillow_rent_data[zillow_rent_data["City"] == "New York"]
    NY_rent_date = NY_rent_date.reset_index(drop=True)
    NY_rent_date_sub = NY_rent_date.drop(columns=['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State','City','Metro','CountyName'])
    column_names = NY_rent_date_sub.columns.tolist()
    for i,j in enumerate(column_names):
        if i > 0:
            rent_month_dict[j] = i-1
    return NY_rent_date_sub

In [21]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    print(f"load zipcode data done")
    #geodf_311_data = download_and_clean_311_data()
    geodf_311_data = load_local_311_data()
    print(f"load 311 data done")
    geodf_tree_data = download_and_clean_tree_data()
    print(f"load trees data done")
    df_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    print(f"load rents data done")
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
# geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
# print(f"load zipcode data done")
# geodf_311_data = download_and_clean_311_data()
# print(f"load 311 data done")

load zipcode data done
Current Time = 02:04:45 Reading from data/erm2-nwe9.json...


In [22]:
df_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)

In [ ]:
# geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
# print(f"load zipcode data done")
# geodf_tree_data = download_and_clean_tree_data()
# print(f"load trees data done")
# df_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
# print(f"load rents data done")

In [ ]:
# geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [13]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   index     248 non-null    int64   
 1   ZIPCODE   248 non-null    object  
 2   geometry  248 non-null    geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 5.9+ KB


In [14]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,index,ZIPCODE,geometry
0,0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [15]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33233422 entries, 0 to 33233421
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   complaint_type  object        
 1   created_date    datetime64[ns]
 2   geometry        geometry      
 3   incident_zip    object        
 4   unique_key      object        
dtypes: datetime64[ns](1), geometry(1), object(3)
memory usage: 1.2+ GB


In [16]:
geodf_311_data.head()

,complaint_type,created_date,geometry,incident_zip,unique_key
0,Derelict Vehicles,2023-11-17 12:00:00,POINT (-73.98863 40.77506),10069,59469711
1,Derelict Vehicles,2023-11-17 12:00:00,POINT (-73.90324 40.75259),11377,59468480
2,Illegal Parking,2023-11-17 01:06:19,POINT (-73.97546 40.59379),11223,59463383
3,Rodent,2023-11-17 01:05:09,POINT (-73.86890 40.83262),10472,59463316
4,Non-Emergency Police Matter,2023-11-17 01:03:42,POINT (-73.86991 40.74827),11373,59462918


In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

In [19]:
def setup_new_postgis_database(username, db_name):
    conn = psycopg2.connect(database=f"{db_name}",
                        host="localhost",
                        user=f"{username}",
                        password="Iris0731",
                        port="5432")
    return conn

In [20]:
conn = setup_new_postgis_database(DB_USER, DB_NAME)

In [21]:
cur = conn.cursor()

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS NYC_zipcodes
(
    id serial PRIMARY KEY,
    zipcode TEXT, 
    geometry geometry(Polygon, 4326),
    CONSTRAINT unique_zipcode UNIQUE (zipcode)
);
"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS NYC_complaints
(
    id INTEGER PRIMARY KEY, 
    created_date date,
    zipcode TEXT,
    complaint_type TEXT,
    location geometry(Point, 4326),
    CONSTRAINT zip_code
        FOREIGN KEY(zipcode)
            REFERENCES NYC_zipcodes(zipcode)
);
"""

NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS NYC_trees
(
    id INTEGER PRIMARY KEY, 
    created_date date,
    zipcode TEXT,
    location geometry,
    species TEXT,
    health TEXT,
    status TEXT,
    CONSTRAINT zip_code
        FOREIGN KEY(zipcode)
            REFERENCES NYC_zipcodes(zipcode)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS NYC_rents
(
    id serial PRIMARY KEY,
    zipcode TEXT,
    rents REAL[],  
    CONSTRAINT zip_code
        FOREIGN KEY(zipcode)
            REFERENCES NYC_zipcodes(zipcode)
);
"""

ZIPCODE_TABLE_INDEX = "CREATE INDEX IF NOT EXISTS zip_geo ON NYC_zipcodes USING GIST(geometry);"

COMLIANTS_TABLE_INDEX = "CREATE INDEX IF NOT EXISTS complaints_location ON NYC_complaints USING GIST(location);"

TREES_TABLE_INDEX = "CREATE INDEX IF NOT EXISTS tree_location ON NYC_trees USING GIST(location);"

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)
    f.write(f"{ZIPCODE_TABLE_INDEX}\n")
    f.write(f"{COMLIANTS_TABLE_INDEX}\n")
    f.write(f"{TREES_TABLE_INDEX}\n")

In [ ]:
# Execute following queries to delete tables if already exits
# cur.execute("DROP TABLE IF EXISTS NYC_trees")
# cur.execute("DROP TABLE IF EXISTS NYC_complaints")
# cur.execute("DROP TABLE IF EXISTS NYC_rents")
# cur.execute("DROP TABLE IF EXISTS NYC_zipcodes")

In [ ]:
# Execute the schema files to create tables
cur.execute(ZIPCODE_SCHEMA)
cur.execute(NYC_311_SCHEMA)
cur.execute(NYC_TREE_SCHEMA)
cur.execute(ZILLOW_SCHEMA)

In [ ]:
cur.execute(ZIPCODE_TABLE_INDEX)
cur.execute(COMLIANTS_TABLE_INDEX)
cur.execute(TREES_TABLE_INDEX)

In [ ]:
conn.commit()

### Add Data to Database


#### Option 1: SQL

In [ ]:
def zipcodes_write_to_table(geodf_zipcode_data):
    for zipcode, geo in zip(geodf_zipcode_data['ZIPCODE'], geodf_zipcode_data['geometry'].to_wkt()): #insert into databse
        query = '''
            INSERT INTO  NYC_zipcodes(zipcode, geometry)
            VALUES 
                (%s, ST_GeomFromText(%s, 4326))
            ''' 
        cur.execute(query, (zipcode, geo))

In [20]:
def complaints_write_to_table(geodf_311_data):
    for id_, created_date, zipcode, complaint_type, geo in zip(geodf_311_data['unique_key'], geodf_311_data['created_date'], geodf_311_data['incident_zip'], geodf_311_data['complaint_type'], geodf_311_data['geometry'].to_wkt()): #insert into databse
        query = '''
            INSERT INTO NYC_complaints(id, created_date, zipcode, complaint_type, location)
            VALUES (%s, %s, %s, %s, ST_SetSRID(ST_GeomFromText(%s), 4326))
            '''
        cur.execute(query, (id_, created_date, zipcode, complaint_type, geo))

In [21]:
complaints_write_to_table(geodf_311_data)

In [ ]:
def trees_write_to_table(geodf_tree_data):
    for id_, created_date, zipcode, specices, health, status, geo in zip(geodf_tree_data['tree_id'], geodf_tree_data['created_at'], 
                                                                         geodf_tree_data['zipcode'], geodf_tree_data['spc_common'], 
                                                                         geodf_tree_data['health'], geodf_tree_data['status'], 
                                                                         geodf_tree_data['the_geom'].to_wkt()): #insert into databse
        query = '''
                INSERT INTO NYC_trees(id, created_date, zipcode, species, health, status, location)
                VALUES (%s, %s, %s, %s, %s, %s, ST_SetSRID(ST_GeomFromText(%s), 4326))
                '''
        cur.execute(query, (id_, created_date, zipcode, specices, health, status, geo))

In [ ]:
def rents_write_to_table(df_zillow_data):
    all_rows_data = []

    for index, row in df_zillow_data.iterrows():
        row_data = row.iloc[1:].tolist()
        all_rows_data.append(row_data)
    
    for zipcode, rent in zip(df_zillow_data['RegionName'], all_rows_data): #insert into databse
        query = '''
                INSERT INTO  NYC_rents(zipcode, rents)
                VALUES (%s, %s)
                '''
        cur.execute(query, (zipcode, rent))

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    zipcodes_write_to_table(tablename_to_dataframe['zipcodes'])
    print("write zipcodes to db done")
    complaints_write_to_table(tablename_to_dataframe['complaints'])
    print("write complaints to db done")
    trees_write_to_table(tablename_to_dataframe['trees'])
    print("write trees to db done")
    rents_write_to_table(tablename_to_dataframe['rents'])
    print("write rents to db done")

In [ ]:
zipcodes_write_to_table(geodf_zipcode_data)
print("write zipcodes to db done")

In [ ]:
trees_write_to_table(geodf_tree_data)
print("write trees to db done")
rents_write_to_table(df_zillow_data)
print("write rents to db done")

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)
conn.commit()

In [ ]:
conn.commit()

In [7]:
# Show the number of entries of each table
cur.execute("SELECT count(*) from NYC_zipcodes")
cur.fetchall()

[(248,)]

In [8]:
# Show a sample entries of each table
cur.execute("SELECT zipcode, LEFT(ST_AsText(geometry), 50) from NYC_zipcodes Limit 2") #count the number of ratios in the database
cur.fetchall()[1]

('11213', 'POLYGON((-73.937398 40.67973,-73.934872 40.679593,')

In [25]:
cur.execute("SELECT count(*) from NYC_complaints")
cur.fetchall()

[(33233422,)]

In [9]:
cur.execute("SELECT id, zipcode, ST_AsText(location) from NYC_complaints Limit 2") #count the number of ratios in the database
cur.fetchall()

[(59469711, '10069', 'POINT(-73.988634 40.775065)'),
 (59468480, '11377', 'POINT(-73.903238 40.752595)')]

In [10]:
cur.execute("SELECT count(*) from NYC_trees")
cur.fetchall()

[(682853,)]

In [11]:
cur.execute("SELECT id, zipcode, ST_AsText(location) from NYC_trees Limit 2") #count the number of ratios in the database
cur.fetchall()

[(180683, '11375', 'POINT(-73.844215 40.723092)'),
 (200540, '11357', 'POINT(-73.818679 40.794111)')]

In [12]:
cur.execute("SELECT count(*) from NYC_rents")
cur.fetchall()

[(145,)]

In [13]:
cur.execute("SELECT id, zipcode, array_to_string(rents, ', ') AS rent FROM NYC_rents;") #count the number of ratios in the database
cur.fetchall()[1]

(2,
 '11208',
 'NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 2334.735, 2372.3096, 2408.2073, 2451.158, 2473.9316, 2498.065, 2479.6006, 2487.125, 2504.0781, 2508.6704, 2588.0303, 2613.7908, 2585.5613, 2633.2007, 2672.0386, 2806.9187, 2765.2244, 2737.5474, 2728.7334')

In [14]:
conn.commit()

## Part 3: Understanding the Data

### Query 1

In [7]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

In [8]:
def write_all_queries_to_file():    
    write_query_to_file(QUERY_1, QUERY_1_FILENAME)
    write_query_to_file(QUERY_2, QUERY_2_FILENAME)
    write_query_to_file(QUERY_3, QUERY_3_FILENAME)
    write_query_to_file(QUERY_4, QUERY_4_FILENAME)
    write_query_to_file(QUERY_5, QUERY_5_FILENAME)
    write_query_to_file(QUERY_6, QUERY_6_FILENAME)

In [9]:
QUERY_1_FILENAME = QUERY_DIR / "Query1_311_complaints_by_zipcodes.sql"
start_date = date(2022, 10, 1)
end_date = date(2023, 9, 30)

QUERY_1 = f'''
    SELECT zipcode, COUNT(*) AS num_complaints
    FROM NYC_complaints
    WHERE created_date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY zipcode
    ORDER BY num_complaints DESC;
'''

In [10]:
QUERY_2_FILENAME = QUERY_DIR / "Query2_top_10_zipcodes_by_trees.sql"

QUERY_2 = '''
        SELECT zipcode, COUNT(*) AS num_trees
        FROM NYC_trees
        GROUP BY zipcode
        ORDER BY num_trees DESC
        LIMIT 10;
'''

In [23]:
QUERY_3_FILENAME = QUERY_DIR / "Query3_Aug_2023_top_10_zipcodes_rents_by_trees.sql"
i = rent_month_dict['2023-08-31']
# Query to find the average rent for each zip code in the top 10 zip codes with the most trees
QUERY_3 = f'''
    SELECT r.zipcode, TO_CHAR(rent_element, '9,999.99') AS avg_rent
    FROM (
        SELECT
            r.zipcode,
            r.rents[{i}] AS rent_element
        FROM
            NYC_rents r

    ) r
    JOIN (
        SELECT zipcode
        FROM NYC_trees
        GROUP BY zipcode
        ORDER BY COUNT(*) DESC
        LIMIT 10
    ) t ON r.zipcode = t.zipcode
    ORDER BY avg_rent DESC;
'''

In [24]:
QUERY_4_FILENAME = QUERY_DIR / "Query4_zipcodes_rents_tress_complaints_correlation_by_rents.sql"
i = rent_month_dict['2023-01-31']
# Query to find the average rent for each zip co de in the top 10 zip codes with the most trees
QUERY_4 = f'''
    SELECT 
        r.zipcode, 
        TO_CHAR(rent_element, '9,999.99') AS avg_rent,
        t.num_trees, 
        c.num_complaints 
    FROM(
        (
            SELECT
                zipcode,
                rents[{i}] AS rent_element
            FROM
                NYC_rents r
            WHERE NOT (rents[96] = 'NaN')
            ORDER BY rents[{i}] 
            LIMIT 5
        ) 
        UNION ALL 
        ( 
            SELECT 
                zipcode,
                rents[{i}] AS rent_element
            FROM
                NYC_rents r
            WHERE NOT (rents[96] = 'NaN')
            ORDER BY rents[{i}] DESC
            LIMIT 5
        )
    )AS r
    JOIN (
        SELECT 
            zipcode, 
            COUNT(*) AS num_trees
        FROM NYC_trees
        GROUP BY zipcode
    ) t ON r.zipcode = t.zipcode
    JOIN (
        SELECT 
            zipcode, 
            COUNT(*) AS num_complaints
        FROM NYC_complaints
        GROUP BY zipcode
    ) c ON r.zipcode = c.zipcode
    ORDER BY avg_rent DESC;
'''


In [25]:
QUERY_5_FILENAME = QUERY_DIR / "Query5_top_10_zipcodes_by_trees_by_geometry_location.sql"
# Query to find the average rent for each zip co de in the top 10 zip codes with the most trees
QUERY_5 = f'''
    SELECT
        z.zipcode,
        COUNT(t.id) AS num_trees
    FROM
        NYC_zipcodes z
    JOIN
        NYC_trees t ON ST_Within(t.location, z.geometry)
    GROUP BY
        z.zipcode
    ORDER BY
        num_trees DESC
    LIMIT 10;
'''

In [26]:
QUERY_6_FILENAME = QUERY_DIR / "Query6_near_compus_trees_infomation.sql"
# Query to find the average rent for each zip co de in the top 10 zip codes with the most trees
QUERY_6 = f'''
    SELECT
        id,
        species,
        health,
        status,
        ST_AsText(location) AS coordinate
    FROM
        NYC_trees
    WHERE
        ST_DWithin(
            location,
            ST_MakePoint(-73.96253174434912, 40.80737875669467)::geography,
            804.672  -- 0.5 mile in meters
        );

'''

In [167]:
write_all_queries_to_file()

In [168]:
conn.commit()

In [27]:
#Execute and display result for query1
cur.execute(QUERY_1)
results = cur.fetchall()
for row in results:
    print(f"Zip Code: {row[0]}, Number of Complaints: {row[1]}")

Zip Code: 11226, Number of Complaints: 49518
Zip Code: 10467, Number of Complaints: 47508
Zip Code: 10468, Number of Complaints: 44201
Zip Code: 11385, Number of Complaints: 44065
Zip Code: 10452, Number of Complaints: 44032
Zip Code: 10457, Number of Complaints: 41638
Zip Code: 10458, Number of Complaints: 41275
Zip Code: 11207, Number of Complaints: 40334
Zip Code: 10456, Number of Complaints: 39919
Zip Code: 11201, Number of Complaints: 39450
Zip Code: 10453, Number of Complaints: 38192
Zip Code: 11208, Number of Complaints: 36905
Zip Code: 10031, Number of Complaints: 33962
Zip Code: 11221, Number of Complaints: 33276
Zip Code: 10466, Number of Complaints: 32684
Zip Code: 10032, Number of Complaints: 32127
Zip Code: 10025, Number of Complaints: 31195
Zip Code: 10462, Number of Complaints: 30866
Zip Code: 11238, Number of Complaints: 30305
Zip Code: 11230, Number of Complaints: 29725
Zip Code: 11368, Number of Complaints: 29425
Zip Code: 11220, Number of Complaints: 29141
Zip Code: 

In [28]:
#Execute and display result for query2
cur.execute(QUERY_2)
results = cur.fetchall()
for row in results:
    print(f"Zip Code: {row[0]}, Number of Trees: {row[1]}")

Zip Code: 10312, Number of Trees: 22186
Zip Code: 10314, Number of Trees: 16905
Zip Code: 10306, Number of Trees: 13030
Zip Code: 10309, Number of Trees: 12650
Zip Code: 11234, Number of Trees: 11253
Zip Code: 11385, Number of Trees: 10937
Zip Code: 11357, Number of Trees: 9449
Zip Code: 11207, Number of Trees: 8634
Zip Code: 11434, Number of Trees: 8274
Zip Code: 11208, Number of Trees: 8245


In [29]:
#Execute and display result for query3
cur.execute(QUERY_3)
results = cur.fetchall()
for row in results:
    print(f"Zip Code:{row[0]}, avg_rent:{row[1]}")

Zip Code:11385, avg_rent: 3,034.41
Zip Code:11207, avg_rent: 3,011.85
Zip Code:11208, avg_rent: 2,765.22
Zip Code:11434, avg_rent: 2,744.81
Zip Code:10314, avg_rent: 2,467.05
Zip Code:11357, avg_rent: 2,417.08
Zip Code:10306, avg_rent: 2,330.52
Zip Code:11234, avg_rent: 2,292.07
Zip Code:10309, avg_rent: 1,733.61
Zip Code:10312, avg_rent: 1,709.74


In [30]:
#Execute and display result for query4
cur.execute(QUERY_4)
results = cur.fetchall()
for row in results:
    print(f"Zip Code:{row[0]}, avg_rent:{row[1]}, number of trees:{row[2]}, number of complaints: {row[3]}")

Zip Code:10007, avg_rent: 7,260.46, number of trees:355, number of complaints: 46393
Zip Code:10282, avg_rent: 7,058.61, number of trees:234, number of complaints: 5948
Zip Code:10013, avg_rent: 5,551.23, number of trees:1202, number of complaints: 149049
Zip Code:10069, avg_rent: 5,003.40, number of trees:119, number of complaints: 6272
Zip Code:10011, avg_rent: 4,789.54, number of trees:2134, number of complaints: 218408
Zip Code:11357, avg_rent: 1,788.94, number of trees:9449, number of complaints: 141854
Zip Code:10462, avg_rent: 1,776.66, number of trees:4234, number of complaints: 302108
Zip Code:10453, avg_rent: 1,762.64, number of trees:3050, number of complaints: 418417
Zip Code:10312, avg_rent: 1,644.01, number of trees:22186, number of complaints: 208250
Zip Code:10309, avg_rent: 1,456.13, number of trees:12650, number of complaints: 114994


In [31]:
#Execute and display result for query5
cur.execute(QUERY_5)
results = cur.fetchall()
for row in results:
    print(f"Zip Code:{row[0]}, number of trees:{row[1]}")

Zip Code:10312, number of trees:22186
Zip Code:10314, number of trees:16905
Zip Code:10306, number of trees:13030
Zip Code:10309, number of trees:12650
Zip Code:11234, number of trees:11253
Zip Code:11385, number of trees:10937
Zip Code:11357, number of trees:9449
Zip Code:11207, number of trees:8634
Zip Code:11434, number of trees:8274
Zip Code:11208, number of trees:8246


In [32]:
#Execute and display result for query6
cur.execute(QUERY_6)
results = cur.fetchall()
for row in results:
    print(f"Tree id:{row[0]}, species:{row[1]}, health:{row[2]}, status:{row[3]}, coordinates:{row[4]}")

Tree id:198514, species:pin oak, health:Good, status:Alive, coordinates:POINT(-73.96208 40.802301)
Tree id:209919, species:London planetree, health:Good, status:Alive, coordinates:POINT(-73.963315 40.808812)
Tree id:209921, species:London planetree, health:Good, status:Alive, coordinates:POINT(-73.963403 40.808745)
Tree id:203887, species:willow oak, health:Good, status:Alive, coordinates:POINT(-73.960719 40.805726)
Tree id:196440, species:American elm, health:Fair, status:Alive, coordinates:POINT(-73.964123 40.811145)
Tree id:209913, species:pin oak, health:Good, status:Alive, coordinates:POINT(-73.963121 40.809077)
Tree id:196605, species:American elm, health:Good, status:Alive, coordinates:POINT(-73.967158 40.807386)
Tree id:196614, species:nan, health:nan, status:Stump, coordinates:POINT(-73.9677 40.806503)
Tree id:188558, species:London planetree, health:Good, status:Alive, coordinates:POINT(-73.960529 40.803618)
Tree id:204127, species:Japanese zelkova, health:Fair, status:Alive,

In [169]:
# Test
!psql --dbname Final_Project -f queries/Query6_near_compus_trees_infomation.sql

   id   |        species         | health | status |         coordinate          
--------+------------------------+--------+--------+-----------------------------
 198514 | pin oak                | Good   | Alive  | POINT(-73.96208 40.802301)
 209919 | London planetree       | Good   | Alive  | POINT(-73.963315 40.808812)
 209921 | London planetree       | Good   | Alive  | POINT(-73.963403 40.808745)
 203887 | willow oak             | Good   | Alive  | POINT(-73.960719 40.805726)
 196440 | American elm           | Fair   | Alive  | POINT(-73.964123 40.811145)
 209913 | pin oak                | Good   | Alive  | POINT(-73.963121 40.809077)
 178550 | Norway maple           | Good   | Alive  | POINT(-73.957399 40.806903)
 189403 | Callery pear           | Fair   | Alive  | POINT(-73.958612 40.807541)
 196606 | honeylocust            | Good   | Alive  | POINT(-73.967199 40.807322)
 198512 | pin oak                | Good   | Alive  | POINT(-73.962306 40.802396)
 196541 | honeylocust      